In [4]:
import pandas as pd
import numpy as np
import datetime as dt

## Step 0 - Declaring the Parameters

In [5]:
# To be used in Profit and Loss calculcation
brokerage = 40

# Quantity of stocks in 1 Lot of Bank Nifty
lot_size = 25

# This is a factor we are including for slippage and other tax deductions
# This variable is to incorporate STT deductions
adjustment_factor = 0.95 

# Interval for which we calucalate our High and Low
interval_dict = {"10 AM" :  45,
                 "11 AM" :  105,
                 "12 PM" :  165,
                 "1 PM"  :  225,
                 "2 PM"  :  285}


interval_in_mins = interval_dict['11 AM']

start_time = '10:59' #change it as per interval selection
end_time = '15:00'

#Target Points
target_param = 100

# Stop Loss Points
loss_param = 175

# This date is an outlier. On this day Trading happens at a different time-frame.
muhurat_trading_day = '04-11-2021' 


#File Path
data_path = '/Users/sreemantakesh/Desktop/The Algotrading Page/Datasets/FUTURES/BANK NIFTY FUT 2021.csv'

## Step 1 - Load the data and perform elementary checks

In [6]:
df = pd.read_csv(data_path)

# Step 2 - Create two columns time, and day. Such that

#### 'Day' = DD-MM-YYYY
#### 'Time' = HH:MM
    
Also, convert, the 'date' column to 'datetime' object.

In [7]:
df['time']= df.date.str.slice(10,16)

df['day'] = df.date.str.slice(0,10)

df["date"] = pd.to_datetime(df["date"])

# Step 3 - At 10:59 AM calculate the day high and day Low  for each day and store them as columns with name as 'interval_high' and 'interval_low'. 

In [8]:
#doing a groupby and only keeping the first 105 entries. Why?
temp = df.groupby('day').head(interval_in_mins)


interval_high_low = temp.groupby('day', as_index=False).agg({"high":"max", "low":"min"})

#renaming the columns
interval_high_low.columns = ['day','interval_high','interval_low']

#Joining with the original dataframe 'df'
df = pd.merge(df, interval_high_low, on = 'day')

# Step 4 - 7

In [9]:
Days = df.day.unique().tolist()

if muhurat_trading_day:

    Days.remove(muhurat_trading_day)

rpt = []

for day in Days:
    
    print('||||||||||||||||||Day -------',day)
    
    #filtering the data for one single day
    d1 = df[df["day"] == day]

    #Storing the 11 AM high and 11 AM low in variables 'interval_high' and 'interval_low'
    interval_high = d1['interval_high'].unique()[0]
    interval_low = d1['interval_low'].unique()[0]

    #Storing the 11 AM high and 11 AM low in variables 'interval_high_time' and 'interval_low_time'
    interval_high_time = d1[d1['high']==interval_high].date.dt.time.values[0]
    interval_low_time = d1[d1['low']==interval_low].date.dt.time.values[0]

    #subsetting the data for post to interval time
    d1 = d1.set_index('date')
    d1 = d1.between_time(start_time,end_time)
    d1 = d1.reset_index()
    
    position = ''

    d1['interval_high_breach'] = np.where(d1.high > interval_high , True, False)
    d1['interval_low_breach'] = np.where(d1.low < interval_low , True, False)


    #Condtion 1 - When after 11 AM both high breach and low breack has happend on that day
    if len(d1[d1['interval_low_breach'] == True]) & len(d1[d1['interval_high_breach'] == True]):
        
        
        long_entry_index = d1[d1['interval_high_breach'] == True].index[0]
        short_entry_index = d1[d1['interval_low_breach'] == True].index[0]


        if long_entry_index < short_entry_index:

            position = 'Long'

            entry_price = interval_high

            entry_index = d1[d1['interval_high_breach'] == True].index[0]

            target    = entry_price + target_param

            stop_loss = entry_price - loss_param

            entry_time = d1['date'].dt.time[entry_index]    


        else:

            position = 'Short'

            entry_price = interval_low

            entry_index = d1[d1['interval_low_breach'] == True].index[0]

            target    = entry_price - target_param

            stop_loss = entry_price + loss_param

            entry_time = d1['date'].dt.time[entry_index]

    #Condtion 2 - When after 11 AM only low breach has happened
    elif len(d1[d1['interval_low_breach'] == True]):

        position = 'Short'

        entry_price = interval_low

        entry_index = d1[d1['interval_low_breach'] == True].index[0]

        target    = entry_price - target_param

        stop_loss = entry_price + loss_param

        entry_time = d1['date'].dt.time[entry_index]

    #Condtion 3 - When after 11 AM only high breach has happened
    elif len(d1[d1['interval_high_breach'] == True]):

        position = 'Long'

        entry_price = interval_high

        entry_index = d1[d1['interval_high_breach'] == True].index[0]

        target    = entry_price + target_param

        stop_loss = entry_price - loss_param

        entry_time = d1['date'].dt.time[entry_index]   

    else:

        position = 'NA'

        entry_price = np.nan

        entry_index = np.nan

        target    = np.nan

        stop_loss = np.nan

        entry_time = np.nan  
        
        
    closing_price = ''
    exit_time = ''

    if position!="NA":

        post_signal_data = d1.iloc[entry_index:,:][["high","low","close","time"]].reset_index(drop=True).copy()        

        #iterate and check if target hits or stop loss hits
        if position == "Long":
            for candle_high, candle_low, closing,time in zip(post_signal_data.high,
                                                             post_signal_data.low,
                                                             post_signal_data.close,
                                                             post_signal_data.time):
                if candle_high > target:
                    result = 'Profit'
                    #print('Profit')
                    closing_price = target
                    exit_time = time
                    break
                elif candle_low < stop_loss:
                    result = 'Loss'
                    #print('Loss')
                    closing_price = stop_loss
                    exit_time = time
                    break
                else:
                    result = "Started but not finished"
                    closing_price = closing
                    exit_time = time

        else:
            for candle_high, candle_low, closing, time in zip(post_signal_data.high,
                                                              post_signal_data.low,
                                                              post_signal_data.close,
                                                              post_signal_data.time):
                if candle_low < target:
                    result = 'Profit'
                    closing_price = target
                    exit_time = time
                    #print('Profit')
                    break
                elif candle_high > stop_loss:
                    result = 'Loss'
                    closing_price = stop_loss
                    exit_time = time
                    #print('Loss')
                    break
                else:
                    result = "Started but not finished"
                    last_price = post_signal_data
                    closing_price = closing
                    exit_time = time
    else:
        result = "NA"
        closing_price = 'NA'



    rpt.append((day,
                interval_high,
                interval_high_time,
                interval_low,
                interval_low_time,
                position,
                entry_time,
                exit_time,
                result,
                entry_price,
                closing_price))        
    
    print("Trade triggered {} position entered ".format(position))
    print("Trade entered at : ",format(entry_time))
    print("Entry Price : ",entry_price)
    print("Traget : ",target)
    print("Stop Loss : ",stop_loss)
    print("Trade Result : ",result)
    print("Trade Exited at : ",exit_time)

||||||||||||||||||Day ------- 04-01-2021
Trade triggered Short position entered 
Trade entered at :  11:00:00
Entry Price :  31200.0
Traget :  31100.0
Stop Loss :  31375.0
Trade Result :  Profit
Trade Exited at :   11:00
||||||||||||||||||Day ------- 05-01-2021
Trade triggered Long position entered 
Trade entered at :  11:05:00
Entry Price :  31296.2
Traget :  31396.2
Stop Loss :  31121.2
Trade Result :  Profit
Trade Exited at :   11:07
||||||||||||||||||Day ------- 06-01-2021
Trade triggered Long position entered 
Trade entered at :  12:18:00
Entry Price :  31971.9
Traget :  32071.9
Stop Loss :  31796.9
Trade Result :  Profit
Trade Exited at :   12:32
||||||||||||||||||Day ------- 07-01-2021
Trade triggered Long position entered 
Trade entered at :  12:30:00
Entry Price :  32229.95
Traget :  32329.95
Stop Loss :  32054.95
Trade Result :  Started but not finished
Trade Exited at :   15:00
||||||||||||||||||Day ------- 08-01-2021
Trade triggered Short position entered 
Trade entered at 

Trade triggered Short position entered 
Trade entered at :  13:11:00
Entry Price :  34975.35
Traget :  34875.35
Stop Loss :  35150.35
Trade Result :  Profit
Trade Exited at :   13:16
||||||||||||||||||Day ------- 17-03-2021
Trade triggered Short position entered 
Trade entered at :  13:50:00
Entry Price :  34651.05
Traget :  34551.05
Stop Loss :  34826.05
Trade Result :  Profit
Trade Exited at :   14:16
||||||||||||||||||Day ------- 18-03-2021
Trade triggered Short position entered 
Trade entered at :  13:00:00
Entry Price :  34462.0
Traget :  34362.0
Stop Loss :  34637.0
Trade Result :  Profit
Trade Exited at :   13:06
||||||||||||||||||Day ------- 19-03-2021
Trade triggered Long position entered 
Trade entered at :  11:00:00
Entry Price :  33952.0
Traget :  34052.0
Stop Loss :  33777.0
Trade Result :  Profit
Trade Exited at :   11:03
||||||||||||||||||Day ------- 22-03-2021
Trade triggered Short position entered 
Trade entered at :  11:00:00
Entry Price :  33775.15
Traget :  33675.15

Trade triggered Long position entered 
Trade entered at :  11:27:00
Entry Price :  34770.0
Traget :  34870.0
Stop Loss :  34595.0
Trade Result :  Profit
Trade Exited at :   12:03
||||||||||||||||||Day ------- 28-05-2021
Trade triggered Short position entered 
Trade entered at :  11:02:00
Entry Price :  35311.9
Traget :  35211.9
Stop Loss :  35486.9
Trade Result :  Profit
Trade Exited at :   12:31
||||||||||||||||||Day ------- 31-05-2021
Trade triggered Long position entered 
Trade entered at :  12:35:00
Entry Price :  35549.0
Traget :  35649.0
Stop Loss :  35374.0
Trade Result :  Started but not finished
Trade Exited at :   15:00
||||||||||||||||||Day ------- 01-06-2021
Trade triggered Short position entered 
Trade entered at :  11:10:00
Entry Price :  35360.0
Traget :  35260.0
Stop Loss :  35535.0
Trade Result :  Loss
Trade Exited at :   13:20
||||||||||||||||||Day ------- 02-06-2021
Trade triggered Long position entered 
Trade entered at :  11:06:00
Entry Price :  35380.0
Traget :  3

Trade triggered NA position entered 
Trade entered at :  nan
Entry Price :  nan
Traget :  nan
Stop Loss :  nan
Trade Result :  NA
Trade Exited at :  
||||||||||||||||||Day ------- 10-08-2021
Trade triggered Short position entered 
Trade entered at :  14:14:00
Entry Price :  36039.95
Traget :  35939.95
Stop Loss :  36214.95
Trade Result :  Started but not finished
Trade Exited at :   15:00
||||||||||||||||||Day ------- 11-08-2021
Trade triggered Short position entered 
Trade entered at :  11:16:00
Entry Price :  35687.35
Traget :  35587.35
Stop Loss :  35862.35
Trade Result :  Loss
Trade Exited at :   12:26
||||||||||||||||||Day ------- 12-08-2021
Trade triggered NA position entered 
Trade entered at :  nan
Entry Price :  nan
Traget :  nan
Stop Loss :  nan
Trade Result :  NA
Trade Exited at :  
||||||||||||||||||Day ------- 13-08-2021
Trade triggered Short position entered 
Trade entered at :  13:38:00
Entry Price :  36056.75
Traget :  35956.75
Stop Loss :  36231.75
Trade Result :  Loss

Traget :  nan
Stop Loss :  nan
Trade Result :  NA
Trade Exited at :  
||||||||||||||||||Day ------- 20-10-2021
Trade triggered Long position entered 
Trade entered at :  11:29:00
Entry Price :  39804.9
Traget :  39904.9
Stop Loss :  39629.9
Trade Result :  Loss
Trade Exited at :   11:49
||||||||||||||||||Day ------- 21-10-2021
Trade triggered Short position entered 
Trade entered at :  11:00:00
Entry Price :  39590.3
Traget :  39490.3
Stop Loss :  39765.3
Trade Result :  Profit
Trade Exited at :   14:25
||||||||||||||||||Day ------- 22-10-2021
Trade triggered Long position entered 
Trade entered at :  11:01:00
Entry Price :  40453.8
Traget :  40553.8
Stop Loss :  40278.8
Trade Result :  Profit
Trade Exited at :   11:11
||||||||||||||||||Day ------- 25-10-2021
Trade triggered Long position entered 
Trade entered at :  11:02:00
Entry Price :  41079.3
Traget :  41179.3
Stop Loss :  40904.3
Trade Result :  Profit
Trade Exited at :   11:24
||||||||||||||||||Day ------- 26-10-2021
Trade trig

In [10]:
rpt_df = pd.DataFrame(rpt,columns= ['day',
                'interval_high',
                'interval_high_time',
                'interval_low',
                'interval_low_time',
                'position',
                'entry_time',
                'exit_time',
                'result',
                'entry_price',
                'closing_price'])

rpt_df

,day,interval_high,interval_high_time,interval_low,interval_low_time,position,entry_time,exit_time,result,entry_price,closing_price
0,04-01-2021,31599.00,09:28:00,31200.00,10:57:00,Short,11:00:00,11:00,Profit,31200.00,31100.0
1,05-01-2021,31296.20,10:54:00,31011.10,09:15:00,Long,11:05:00,11:07,Profit,31296.20,31396.2
2,06-01-2021,31971.90,09:51:00,31798.25,09:15:00,Long,12:18:00,12:32,Profit,31971.90,32071.9
3,07-01-2021,32229.95,09:34:00,32040.15,10:26:00,Long,12:30:00,15:00,Started but not finished,32229.95,32121.2
4,08-01-2021,32337.95,09:15:00,32131.00,09:42:00,Short,13:39:00,15:00,Started but not finished,32131.00,32210.0
...,...,...,...,...,...,...,...,...,...,...,...
240,24-12-2021,35314.95,09:15:00,34614.00,10:35:00,NA,NaN,,NA,NaN,NA
241,27-12-2021,34843.40,10:47:00,34264.35,09:21:00,Long,11:47:00,12:20,Profit,34843.40,34943.4
242,28-12-2021,35375.00,09:15:00,35050.00,10:31:00,NA,NaN,,NA,NaN,NA
243,29-12-2021,35398.00,09:50:00,35045.00,09:15:00,NA,NaN,,NA,NaN,NA


In [11]:
rpt_df['new_date'] = pd.to_datetime(rpt_df.day).dt.strftime('%d-%m-%Y')
rpt_df['month_name'] = pd.to_datetime(rpt_df.new_date).dt.month_name()


/var/folders/b5/722v8gw92z518zxsqz5sfskh0000gn/T/ipykernel_64979/2467905188.py:1: UserWarning: Parsing '13-01-2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  rpt_df['new_date'] = pd.to_datetime(rpt_df.day).dt.strftime('%d-%m-%Y')
/var/folders/b5/722v8gw92z518zxsqz5sfskh0000gn/T/ipykernel_64979/2467905188.py:1: UserWarning: Parsing '14-01-2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  rpt_df['new_date'] = pd.to_datetime(rpt_df.day).dt.strftime('%d-%m-%Y')
/var/folders/b5/722v8gw92z518zxsqz5sfskh0000gn/T/ipykernel_64979/2467905188.py:1: UserWarning: Parsing '15-01-2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  rpt_df['new_date'] = pd.to_datetime(rpt_df.day).dt.strftime('%d-%m-%Y')
/var/folders/b5/722v8gw92z518zxsqz5sfskh0000gn/T/ipykernel_64979/2467905188.py:1: UserWarning: Parsing '18-01-2021' in DD/MM/YYYY for